# Projet Arbres de décision et Méthodes d'agrégations
-- Par Isaline Hervé - M2 ECAP --

# Importation des packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer, SimpleImputer
import scipy.stats as stats

# Importation des données et premiers traitements
## Importation des données

In [7]:
df = pd.read_csv('./data/train.csv', header=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7094 entries, 0 to 7093
Data columns (total 25 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   nom_du_projet                        7094 non-null   object 
 1   resume_du_projet                     6917 non-null   object 
 2   montant_engage                       7094 non-null   float64
 3   numero_ej                            6025 non-null   float64
 4   numero_operateur                     1072 non-null   object 
 5   opérateur                            1072 non-null   object 
 6   demarche                             6962 non-null   object 
 7   nom_region                           6929 non-null   object 
 8   nom_departement                      6929 non-null   object 
 9   code_departement                     6929 non-null   object 
 10  siret_beneficiaire                   6930 non-null   float64
 11  raison_sociale_beneficiaire   

In [9]:
df.head()

,nom_du_projet,resume_du_projet,montant_engage,numero_ej,numero_operateur,opérateur,demarche,nom_region,nom_departement,code_departement,...,siret,activitePrincipale,codePostal,dateCreation,categorieEntreprise,trancheEffectifsUniteLegale,societeMissionUniteLegale,etatAdministratifEtablissement,anneeEffectifsEtablissement,economieSocialeSolidaireUniteLegale
0,"Changement des fenêtres ,des portes et pose de...","Suite aux différents audits, il convient de fi...",20710.00,2104489041,NaN,NaN,Rénovation énergétique des bâtiments publics l...,Normandie,Eure,27,...,20005607500019,84.11Z,27160,2016-01-01,PME,12,NaN,A,2023,N
1,ZAC Bernard Duval,La ZAC Claude Bernard / Alexandre Duval à Renn...,200000.00,2104468899,NaN,NaN,Recyclage foncier,Bretagne,Ille-et-Vilaine,35,...,52318955300010,42.99Z,35200,2010-05-10,ETI,03,NaN,A,2023,N
2,Travaux d'urgence de confortement et d'aménage...,Suites aux intempéries du début du mois de déc...,14157.02,2104385704,NaN,NaN,Appui aux collectivités de montagne soumises à...,Provence-Alpes-Côte d'Azur,Hautes-Alpes,05,...,20006732000016,84.11Z,5230,2017-01-01,PME,12,NaN,A,2023,N
3,Rénovation thermique de l'école de GERBEPAL - ...,La commune de GERBÉPAL souhaite rénover le bât...,202019.00,2104371080,NaN,NaN,Rénovation énergétique des bâtiments publics l...,Grand Est,Vosges,88,...,21880198300105,84.11Z,88430,1999-12-25,PME,11,NaN,A,2023,N
4,Rénovation énergétique du Centre Omnisport de ...,Situé à proximité du quartier de Marbé au nord...,587251.00,2104408744,NaN,NaN,Rénovation énergétique des bâtiments publics l...,Bourgogne-Franche-Comté,Saône-et-Loire,71,...,21710270600017,84.11Z,71000,1983-03-01,ETI,42,NaN,A,2023,N


In [11]:
# Modification des noms de colonnes
df = df.rename(columns={"opérateur": "operateur", "forme juridique_beneficiaire": "forme_juridique_beneficiaire"})

In [ ]:
# Conversion des colonnes numériques en string
cols_to_string = [
    "numero_ej",
    "siret_beneficiaire",
    "code_commune", 
    "siret",
    "codePostal",
]

for col in cols_to_string:
    df[col] = df[col].astype("Int64").astype("string")

# Conversion des colonnes avec nombre entier en Int64
cols_to_int = [
    "anneeEffectifsEtablissement"
]

for col in cols_to_int:
    df[col] = df[col].astype("Int64") # Int64 pour la gestion des NaN


# Conversion des colonnes objects en catégorielles
categorical_cols = [
    "operateur",
    "nom_region",
    "nom_departement",
    "code_departement",
    "forme_juridique_beneficiaire",
    "activitePrincipale",
    "categorieEntreprise",
    "trancheEffectifsUniteLegale",
    "societeMissionUniteLegale",
    "etatAdministratifEtablissement",
    "economieSocialeSolidaireUniteLegale",
]

df[categorical_cols] = df[categorical_cols].astype("category")

# Conversion des variables date en format datetime
df["dateCreation"] = pd.to_datetime(df["dateCreation"], errors="coerce")

# Variables textes
text_cols = [
    "nom_du_projet",
    "resume_du_projet",
    "demarche",
    "raison_sociale_beneficiaire",
]

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7094 entries, 0 to 7093
Data columns (total 25 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   nom_du_projet                        7094 non-null   object        
 1   resume_du_projet                     6917 non-null   object        
 2   montant_engage                       7094 non-null   float64       
 3   numero_ej                            6025 non-null   string        
 4   numero_operateur                     1072 non-null   object        
 5   opérateur                            1072 non-null   object        
 6   demarche                             6962 non-null   category      
 7   nom_region                           6929 non-null   category      
 8   nom_departement                      6929 non-null   category      
 9   code_departement                     6929 non-null   category      
 10  siret_benefi

In [3]:
df.describe()

,montant_engage,numero_ej,siret_beneficiaire,siret,codePostal,anneeEffectifsEtablissement
count,7.094000e+03,6.025000e+03,6.930000e+03,6.929000e+03,6926.000000,6700.0
mean,1.818024e+05,2.103957e+09,2.416677e+13,2.416718e+13,47214.573058,2023.0
std,3.776175e+05,1.940204e+07,1.103018e+13,1.103092e+13,26992.019802,0.0
min,2.875000e+02,1.000184e+09,6.380158e+11,6.380158e+11,1000.000000,2023.0
25%,1.811075e+04,2.104365e+09,2.112017e+13,2.112017e+13,25027.500000,2023.0
50%,6.324140e+04,2.104426e+09,2.143009e+13,2.143009e+13,44205.000000,2023.0
75%,2.006882e+05,2.104508e+09,2.188046e+13,2.188047e+13,69205.000000,2023.0
max,1.160000e+07,2.104601e+09,9.976258e+13,9.976258e+13,98840.000000,2023.0


## Vérification et traitement des doublons

In [4]:
# Vérification des doublons 
dupes = df.duplicated().sum()
print(f"Number of duplicate rows: {dupes}")
# Pas de doublons détectés

Number of duplicate rows: 0


## Vérification et traitement des valeurs manquantes

In [ ]:
# Vérification des valeurs manquantes
print(f"Number of missing values per column:\n{df.isnull().sum()}")
# Beaucoup de valeurs manquantes, sauf dans nom_du_projet et montant_engage

Number of missing values per column:
nom_du_projet                             0
resume_du_projet                        177
montant_engage                            0
numero_ej                              1069
numero_operateur                       6022
opérateur                              6022
demarche                                132
nom_region                              165
nom_departement                         165
code_departement                        165
siret_beneficiaire                      164
raison_sociale_beneficiaire             138
forme juridique_beneficiaire            138
code_commune                            132
nom_commune                             132
siret                                   165
activitePrincipale                      165
codePostal                              168
dateCreation                            165
categorieEntreprise                     218
trancheEffectifsUniteLegale             165
societeMissionUniteLegale              

In [ ]:
# En pourcentage
na = df.isna().sum()/df.shape[0]*100 # pourcentage de valeurs manquantes par variable
palettehex = sns.color_palette("Paired").as_hex()
px.bar(na.sort_values(),
         title = "Pourcentage de valeurs manquantes par variable",
         color_discrete_sequence = palettehex[0:1],
         template='ggplot2',
         range_y = [0,100],
         width=800,
         labels={'value':'%', 'index':''},
         color=None
        ).update_traces(showlegend=False).update_xaxes(tickangle=45)

In [ ]:

class MissingValuesProcessor:
    """
    Classe pour le traitement des valeurs manquantes dans un DataFrame pandas.

    - Supprime les colonnes avec plus de `missing_threshold` de valeurs manquantes
    - Impute :
        * numériques : IterativeImputer (MICE)
        * catégorielles / string : valeur la plus fréquente
        * datetime : valeur la plus fréquente
    """

    def __init__(self, missing_threshold=0.30, random_state=0, max_iter=10):
        self.missing_threshold = missing_threshold
        self.random_state = random_state
        self.max_iter = max_iter

        self.dropped_columns_ = None
        self.num_cols_ = None
        self.cat_cols_ = None
        self.datetime_cols_ = None

        self.imputer_num_ = None
        self.imputer_cat_ = None
        self.imputer_datetime_ = None

    def fit(self, df: pd.DataFrame):
        # 1) Suppression des colonnes trop manquantes
        missing_ratio = df.isna().mean()
        self.dropped_columns_ = missing_ratio[
            missing_ratio > self.missing_threshold
        ].index.tolist()

        df_reduced = df.drop(columns=self.dropped_columns_)

        # 2) Détection des types
        self.num_cols_ = df_reduced.select_dtypes(
            include=["int64", "float64", "Int64"]
        ).columns.tolist()

        self.datetime_cols_ = df_reduced.select_dtypes(
            include=["datetime64[ns]", "datetime64"]
        ).columns.tolist()

        self.cat_cols_ = df_reduced.select_dtypes(
            include=["object", "category", "string"]
        ).columns.tolist()

        # 3) Initialisation des imputers
        if self.num_cols_:
            self.imputer_num_ = IterativeImputer(
                max_iter=self.max_iter,
                random_state=self.random_state
            )
            self.imputer_num_.fit(df_reduced[self.num_cols_])

        if self.cat_cols_:
            self.imputer_cat_ = SimpleImputer(strategy="most_frequent")
            self.imputer_cat_.fit(df_reduced[self.cat_cols_])

        if self.datetime_cols_:
            self.imputer_datetime_ = SimpleImputer(strategy="most_frequent")
            self.imputer_datetime_.fit(df_reduced[self.datetime_cols_])

        return self

    def transform(self, df: pd.DataFrame) -> pd.DataFrame:
        # Suppression des colonnes trop manquantes
        df_reduced = df.drop(columns=self.dropped_columns_)

        dfs = []

        # Numériques
        if self.num_cols_:
            num_imputed = self.imputer_num_.transform(df_reduced[self.num_cols_])
            df_num = pd.DataFrame(
                num_imputed,
                columns=self.num_cols_,
                index=df_reduced.index
            )
            dfs.append(df_num)

        # Catégorielles / string
        if self.cat_cols_:
            cat_imputed = self.imputer_cat_.transform(df_reduced[self.cat_cols_])
            df_cat = pd.DataFrame(
                cat_imputed,
                columns=self.cat_cols_,
                index=df_reduced.index
            )
            dfs.append(df_cat)

        # Datetime
        if self.datetime_cols_:
            dt_imputed = self.imputer_datetime_.transform(
                df_reduced[self.datetime_cols_]
            )
            df_dt = pd.DataFrame(
                dt_imputed,
                columns=self.datetime_cols_,
                index=df_reduced.index
            )
            # reconversion explicite en datetime
            for col in self.datetime_cols_:
                df_dt[col] = pd.to_datetime(df_dt[col], errors="coerce")

            dfs.append(df_dt)

        # Reconstruction finale
        df_imputed = pd.concat(dfs, axis=1)

        # Remettre l'ordre initial des colonnes restantes
        df_imputed = df_imputed[df_reduced.columns]

        return df_imputed

    def fit_transform(self, df: pd.DataFrame) -> pd.DataFrame:
        self.fit(df)
        return self.transform(df)


In [ ]:
imputer = Nettoyeur_perso(col_a_retirer=['surgery', 'hospital_number', 'outcome', 'lesion_1', 
    'lesion_2', 'lesion_3', 'cp_data'],
    seuil=0.6)
# Apprentissage des colonnes à supprimer et des valeurs de remplacement, puis application du nettoyage
horsesClean = imputer.fit_transform(horses)
print(horsesClean.shape)
horsesClean.head()

## Création de nouvelles variables


# Statistiques descriptives
## Statistiques descriptives univariées

In [ ]:
# Distribution

In [ ]:
# Boxplot - Valeurs extrêmes

## Statistiques descriptives bivariées

In [ ]:
# Corrélation entre variables explicatives

In [ ]:
# Corrélation entre variables explicatives et variable cible

# Modélisation